<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/jvcqp2iy2jlx2b32rmzdt0tx8lvxgzkp.png" width = 300, align = "center"></a>


# <center> Text generation using RNN/LSTM (Character-level)</center>

<div class="alert alert-block alert-info">
<font size = 3><strong>In this notebook you will learn the How to use TensorFlow for create a Recurrent Neural Network</strong></font>
<br>    
- <a href="#intro">Introduction</a>
<br>
- <p><a href="#arch">Architectures</a></p>
    - <a href="#lstm">Long Short-Term Memory Model (LSTM)</a>

- <p><a href="#build">Building a LSTM with TensorFlow</a></p>
</div>
----------------

This code implements a Recurrent Neural Network with LSTM/RNN units for training/sampling from character-level language models. In other words, the model takes a text file as input and trains the RNN network that learns to predict the next character in a sequence.  
The RNN can then be used to generate text character by character that will look like the original training data. 

This code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and the code is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).




First, import the requiered libraries:

In [1]:
import tensorflow as tf
import time
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np
#from tensorflow.python.ops import rnn_cell
#from tensorflow.python.ops import seq2seq

Couldn't import dot_parser, loading of dot files will not be possible.


### Data loader
The following cell is a class that help to read data from input file.

In [2]:
class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "input.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    def preprocess(self, input_file, vocab_file, tensor_file):
        with codecs.open(input_file, "r", encoding=self.encoding) as f:
            data = f.read()
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
        np.save(tensor_file, self.tensor)

    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        # When the data (tensor) is too small, let's give them a better error message
        if self.num_batches==0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)


    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

### Parameters
#### Batch, number_of_batch, batch_size and seq_length
what is batch, number_of_batch, batch_size and seq_length in the charcter level example?  

Lets assume the input is this sentence: '__here is an example__'. Then:
- txt_length = 18  
- seq_length = 3  
- batch_size = 2  
- number_of_batchs = 18/3*2 = 3
- batch = array (['h','e','r'],['e',' ','i'])
- sample Seq = 'her'  

Ok, now, lets look at a real dataset, with real parameters. 

In [3]:
seq_length = 50 # RNN sequence length
batch_size = 60  # minibatch size, i.e. size of data in each epoch
num_epochs = 125 # you should change it to 50 if you want to see a relatively good results
learning_rate = 0.002
decay_rate = 0.97
rnn_size = 128 # size of RNN hidden state (output dimension)
num_layers = 2 #number of layers in the RNN

We download the input file, and print a part of it:

In [4]:
!wget -nv -O input.txt https://ibm.box.com/shared/static/a3f9e9mbpup09toq35ut7ke3l3lf03hg.txt 
with open('input.txt', 'r') as f:
    read_data = f.read()
    print read_data[0:100]
f.closed

2018-10-28 09:04:24 URL:https://public.boxcloud.com/d/1/b1!3rUB8-IPs5lMteO-NAe8WUgJo2ttzLBxCotAIWioeQkyUnUf0auzLR7p31101AUKxW8P62XI7nBrCFqU39I7mJHF4_lnX8h58AJZH94YTHUDLkb6U2wAN5uKl1FtcD-_4gfHtKtIOZ5p5ndo8SRO9-Qrqb9ZwWuPNdlsP0tinrqEUD8gwc_Gh5z8DJ_luo1zZeSJlDWK48FmLYgFA_3wEYnDQxP_yVBSlk7zdMD-4v37MPYv4dxgK8QfsCOVG7nhdyy1hWyRUncursN-crro34bTdw_7CGDhYqfEGCMPe6kb2Ky8QSHcQnJdv0tSuKryqJmWjPP6IJG0zHIxo7sTNL95aNi3aG58goWCsLSNGcCLuysh-oySToiFDMBRUmCDPZVufZPhfBXJc-ZaP82Y5TYo2T4027Zz5Qxqox45OYnHP-kH6elea5GWBAzOIvuC0qWb5uW64IUu0IWswxiZ-eFmQvhUeEd7styDfv5UqGsPgtEQ4hBDSTR4mLm70SGCsbBsKW7vD1GIycAiLKULjC2dGFM-qGDWaxZ2-XHMMz3U3NYNUQpLowoIAcVf_gLZQJVJ5tocv7wF0TXW6jwfTnrwc-1PPJwGwFaro8s0VZTQcpPj32qJAh5_uDhg_5Vxa9RIqwuEqm9my9CtQuJlOrvAPwt13ziyb9FFLN-eb_GSLHu0l3cgbgVZSn7a7DczV45dBS2qLRP6so5Mc93ouR7e-zK7m8xJ7br1E3Vz2zcG5EhpZO-EKeHilzQEKJ5GLSu3sPSxiyj9hzsWZnvAagXFAoSn7n5C8F5McSgwp4AxeXPO58ngONfj8l78CddXIIQXnBhOCzJdXTn2kxrNU363rNh9tRO2hhi91NfO7PzEu-jlnMznZoT3odVeO7zTCV7oCyO7P-eMnLn-4vXofnKdhsbB8Mq5vBBd9ahHtNlnB

True

Now, we can read the data at batches using the __TextLoader__ class. It will convert the characters to numbers, and represent each sequence as a vector in batches:

In [5]:
data_loader = TextLoader('', batch_size, seq_length)
vocab_size = data_loader.vocab_size
print "vocabulary size:" ,data_loader.vocab_size
print "Characters:" ,data_loader.chars
print "vocab number of 'F':",data_loader.vocab['F']
print "Character sequences (first batch):", data_loader.x_batches[0]

loading preprocessed files
vocabulary size: 65
Characters: (u' ', u'e', u't', u'o', u'a', u'h', u's', u'r', u'n', u'i', u'\n', u'l', u'd', u'u', u'm', u'y', u',', u'w', u'f', u'c', u'g', u'I', u'b', u'p', u':', u'.', u'A', u'v', u'k', u'T', u"'", u'E', u'O', u'N', u'R', u'S', u'L', u'C', u';', u'W', u'U', u'H', u'M', u'B', u'?', u'G', u'!', u'D', u'-', u'F', u'Y', u'P', u'K', u'V', u'j', u'q', u'x', u'z', u'J', u'Q', u'Z', u'X', u'3', u'&', u'$')
vocab number of 'F': 49
Character sequences (first batch): [[49  9  7 ...  1  4  7]
 [19  4 14 ... 14  9 20]
 [ 8 20 10 ...  8 10 18]
 ...
 [21  2  0 ...  0 21  0]
 [ 9  7  7 ...  0  2  3]
 [ 3  7  0 ...  5  9 23]]


### Input and output

In [6]:
x,y = data_loader.next_batch()
x

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]])

In [7]:
x.shape  #batch_size =60, seq_length=50

(60, 50)

Here, __y__ is the next character for each character in __x__:

In [8]:
y

array([[ 9,  7,  6, ...,  4,  7,  0],
       [ 4, 14, 22, ...,  9, 20,  5],
       [20, 10, 29, ..., 10, 18,  4],
       ...,
       [ 2,  0,  6, ..., 21,  0,  6],
       [ 7,  7,  4, ...,  2,  3,  0],
       [ 7,  0, 33, ...,  9, 23,  0]])

### LSTM Architecture
Each LSTM cell has 5 parts:
1. Input
2. prv_state
3. prv_output
4. new_state
5. new_output


- Each LSTM cell has an input layre, which its size is 128 units in our case. The input vector's dimension also is 128, which is the dimensionality of embedding vector, so called, dimension size of W2V/embedding, for each character/word.
- Each LSTM cell has a hidden layer, where there are some hidden units. The argument n_hidden=128 of BasicLSTMCell is the number of hidden units of the LSTM (inside A). It keeps the size of the output and state vector. It is also known as, rnn_size, num_units, num_hidden_units, and LSTM size
- An LSTM keeps two pieces of information as it propagates through time: 
    - __hidden state__ vector: Each LSTM cell accept a vector, called __hidden state__ vector, of size n_hidden=128, and its value is returned to the LSTM cell in the next step. The __hidden state__ vector; which is the memory of the LSTM, accumulates using its (forget, input, and output) gates through time. "num_units" is equivalant to "size of RNN hidden state". number of hidden units is the dimensianality of the output (= dimesianality of the state) of the LSTM cell.
    - __previous time-step output__: For each LSTM cell that we initialize, we need to supply a value (128 in this case) for the hidden dimension, or as some people like to call it, the number of units in the LSTM cell. 


#### num_layers = 2 
- number of layers in the RNN, is defined by num_layers
- An input of MultiRNNCell is __cells__ which is list of RNNCells that will be composed in this order.

### Defining stacked RNN Cell

__BasicRNNCell__ is the most basic RNN cell.

In [9]:
cell = tf.contrib.rnn.BasicRNNCell(rnn_size)

In [10]:
# a two layer cell
stacked_cell = tf.contrib.rnn.MultiRNNCell([cell] * num_layers)

In [11]:
# hidden state size
stacked_cell.output_size

128

__state__ varibale keeps output and new_state of the LSTM, so it is a touple of size:

In [12]:
stacked_cell.state_size

(128, 128)

Lets define input data:

In [13]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])# a 60x50
input_data

<tf.Tensor 'Placeholder:0' shape=(60, 50) dtype=int32>

and target data:

In [14]:
targets = tf.placeholder(tf.int32, [batch_size, seq_length]) # a 60x50
targets

<tf.Tensor 'Placeholder_1:0' shape=(60, 50) dtype=int32>

The memory state of the network is initialized with a vector of zeros and gets updated after reading each character.

__BasicRNNCell.zero_state(batch_size, dtype)__ Return zero-filled state tensor(s). In this function, batch_size
representing the batch size.

In [15]:
initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size ? 60x128

Lets check the value of the input_data again:

In [16]:
session = tf.Session()
feed_dict={input_data:x, targets:y}
session.run(input_data, feed_dict)

array([[49,  9,  7, ...,  1,  4,  7],
       [19,  4, 14, ..., 14,  9, 20],
       [ 8, 20, 10, ...,  8, 10, 18],
       ...,
       [21,  2,  0, ...,  0, 21,  0],
       [ 9,  7,  7, ...,  0,  2,  3],
       [ 3,  7,  0, ...,  5,  9, 23]], dtype=int32)

### Embedding
In this section, we build a 128-dim vector for each character. As we have 60 batches, and 50 character in each sequence, it will generate a [60,50,128] matrix.

__Notice:__ The function `tf.get_variable()` is used to share a variable and to initialize it in one place. `tf.get_variable()` is used to get or create a variable instead of a direct call to `tf.Variable`. 

In [17]:
with tf.variable_scope('rnnlm', reuse=False):
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
    softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65)
    #with tf.device("/cpu:0"):
        
    # embedding variable is initialized randomely
    embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128

    # embedding_lookup goes to each row of input_data, and for each character in the row, finds the correspond vector in embedding
    # it creates a 60*50*[1*128] matrix
    # so, the first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character
    em = tf.nn.embedding_lookup(embedding, input_data) # em is 60x50x[1*128]
    # split: Splits a tensor into sub tensors.
    # syntax:  tf.split(split_dim, num_split, value, name='split')
    # it will split the 60x50x[1x128] matrix into 50 matrix of 60x[1*128]
    inputs = tf.split(em, seq_length, 1)
    # It will convert the list to 50 matrix of [60x128]
    inputs = [tf.squeeze(input_, [1]) for input_ in inputs]

Lets take a look at the __embedding__, __em__, and __inputs__ variabbles:

Embedding variable is initialized with random values:

In [18]:
session.run(tf.global_variables_initializer())
#print embedding.shape
session.run(embedding)

array([[ 0.03954738,  0.17300512, -0.10137776, ..., -0.062298  ,
        -0.0941056 , -0.12903407],
       [-0.12152613,  0.1390057 , -0.16910382, ...,  0.1526499 ,
         0.05287218, -0.01532654],
       [ 0.1230929 , -0.00960788, -0.02973355, ..., -0.05113326,
         0.1237634 ,  0.04454418],
       ...,
       [ 0.02798395, -0.00973131,  0.02267058, ...,  0.10103364,
         0.01064028,  0.16067521],
       [-0.06235244, -0.12259279,  0.0587059 , ..., -0.07476767,
        -0.00902352, -0.16558826],
       [-0.02394016,  0.03548509,  0.09613512, ..., -0.09796036,
        -0.13561828, -0.02664728]], dtype=float32)

The first elemnt of em, is a matrix of 50x128, which each row of it is vector representing that character

In [19]:
em = tf.nn.embedding_lookup(embedding, input_data)
emp = session.run(em,feed_dict={input_data:x})
print emp.shape
emp[0]

(60, 50, 128)


array([[-0.04043253, -0.00378796, -0.16874272, ..., -0.00976548,
        -0.16098472,  0.0991527 ],
       [ 0.02144207, -0.10303521,  0.01491348, ...,  0.10539491,
         0.07671736, -0.14640948],
       [-0.11909426,  0.08188854, -0.03067327, ...,  0.08367483,
        -0.03588931,  0.1647252 ],
       ...,
       [-0.12152613,  0.1390057 , -0.16910382, ...,  0.1526499 ,
         0.05287218, -0.01532654],
       [ 0.13002573,  0.01667233, -0.11037925, ..., -0.02999684,
        -0.01412289,  0.04552826],
       [-0.11909426,  0.08188854, -0.03067327, ...,  0.08367483,
        -0.03588931,  0.1647252 ]], dtype=float32)

Let's consider each sequence as a sentence of length 50 characters, then, the first item in __inputs__ is a [60x128] vector which represents the first characters of 60 sentences.

In [20]:
inputs = tf.split(em, seq_length, 1)
inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
inputs[0:5]

[<tf.Tensor 'Squeeze:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_2:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'Squeeze_4:0' shape=(60, 128) dtype=float32>]

### Feeding a batch of 50 sequence to a RNN:

The feeding process for iputs is as following:

- Step 1:  first character of each of the 50 sentences (in a batch) is entered in parallel.  
- Step 2:  second character of each of the 50 sentences is input in parallel. 
- Step n: nth character of each of the 50 sentences is input in parallel.  

The parallelism is only for efficiency.  Each character in a batch is handled in parallel,  but the network sees one character of a sequence at a time and does the computations accordingly. All the computations involving the characters of all sequences in a batch at a given time step are done in parallel. 

In [21]:
session.run(inputs[0],feed_dict={input_data:x})

array([[-0.04043253, -0.00378796, -0.16874272, ..., -0.00976548,
        -0.16098472,  0.0991527 ],
       [ 0.14231332,  0.02843131, -0.12345245, ..., -0.10547654,
        -0.09614094, -0.10258756],
       [-0.17053485,  0.13065504,  0.01885459, ..., -0.07024419,
         0.03325306, -0.06304   ],
       ...,
       [-0.04979004,  0.03105161, -0.07664049, ...,  0.01209794,
         0.04466297, -0.09416938],
       [ 0.02144207, -0.10303521,  0.01491348, ...,  0.10539491,
         0.07671736, -0.14640948],
       [-0.00180891,  0.15713881,  0.01456305, ...,  0.01076694,
         0.0962152 , -0.00291556]], dtype=float32)

Feeding the RNN with one batch, we can check the new output and new state of network:

In [22]:
#outputs is 50x[60*128]
outputs, new_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, initial_state, stacked_cell, loop_function=None, scope='rnnlm')
new_state

(<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_98:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_99:0' shape=(60, 128) dtype=float32>)

In [23]:
outputs[0:5]

[<tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_1:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_3:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_5:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_7:0' shape=(60, 128) dtype=float32>,
 <tf.Tensor 'rnnlm_1/rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/Tanh_9:0' shape=(60, 128) dtype=float32>]

Let's check the output of network after feeding it with first batch:

In [24]:
first_output = outputs[0]
session.run(tf.global_variables_initializer())
session.run(first_output,feed_dict={input_data:x})

array([[-0.0627531 ,  0.00183832,  0.10754857, ...,  0.01956914,
         0.03474949,  0.10239247],
       [ 0.12695885, -0.01237517, -0.06931474, ..., -0.02350149,
         0.07140766, -0.10719743],
       [ 0.02518672,  0.12954985,  0.06966894, ..., -0.09315084,
         0.06086245,  0.07421201],
       ...,
       [ 0.05102353, -0.01884814, -0.0762753 , ..., -0.02305681,
        -0.03018404, -0.07961081],
       [ 0.00977529, -0.04077787,  0.13655712, ..., -0.05319509,
        -0.14298522,  0.09114654],
       [ 0.02268049, -0.06146167,  0.05628747, ..., -0.03728609,
        -0.03686199, -0.03544486]], dtype=float32)

As it was explained, __outputs__ variable is a 50x[60x128] tensor. We need to reshape it back to [60x50x128] to be able to calculate the probablity of the next character using the softmax. The __softmax_w__ shape is [rnn_size, vocab_size],whihc is [128x65] in our case. Threfore, we have a fully connected layer on top of LSTM cells, which help us to decode the next charachter. We can use the __softmax(output * softmax_w + softmax_b)__ for this purpose. The shape of the matrixis would be:

softmax([60x50x128]x[128x65]+[1x65]) = [60x50x65]

We can do it step-by-step:

In [25]:
output = tf.reshape(tf.concat( outputs,1), [-1, rnn_size])
output

<tf.Tensor 'Reshape:0' shape=(3000, 128) dtype=float32>

In [26]:
logits = tf.matmul(output, softmax_w) + softmax_b
logits

<tf.Tensor 'add:0' shape=(3000, 65) dtype=float32>

In [27]:
probs = tf.nn.softmax(logits)
probs

<tf.Tensor 'Softmax:0' shape=(3000, 65) dtype=float32>

Here is the probablity of the next chracter in all batches:

In [28]:
session.run(tf.global_variables_initializer())
session.run(probs,feed_dict={input_data:x})

array([[0.01593737, 0.01780769, 0.01899737, ..., 0.01267415, 0.01525521,
        0.01662062],
       [0.01555159, 0.01579914, 0.01528462, ..., 0.01292581, 0.01710688,
        0.02083142],
       [0.01250922, 0.01745282, 0.0177651 , ..., 0.01427494, 0.02199795,
        0.01762353],
       ...,
       [0.01928865, 0.02350534, 0.01721064, ..., 0.01174289, 0.01965896,
        0.01755373],
       [0.01546535, 0.01691773, 0.01417662, ..., 0.01347447, 0.01358752,
        0.02524465],
       [0.01386683, 0.01397614, 0.01667909, ..., 0.01492867, 0.01830796,
        0.02122142]], dtype=float32)

Now, we are in the position to calculate the cost of training with __loss function__, and keep feedng the network to learn it. But, the question is: what the LSTM networks learn?

In [29]:
grad_clip =5.
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'rnnlm/softmax_w:0' shape=(128, 65) dtype=float32_ref>,
 <tf.Variable 'rnnlm/softmax_b:0' shape=(65,) dtype=float32_ref>,
 <tf.Variable 'rnnlm/embedding:0' shape=(65, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/kernel:0' shape=(256, 128) dtype=float32_ref>,
 <tf.Variable 'rnnlm/multi_rnn_cell/cell_0/basic_rnn_cell/bias:0' shape=(128,) dtype=float32_ref>]

# All together
Now, let's put all of parts together in a class, and train the model:

In [30]:
class LSTMModel():
    def __init__(self,sample=False):
        rnn_size = 128 # size of RNN hidden state vector
        batch_size = 60 # minibatch size, i.e. size of dataset in each epoch
        seq_length = 50 # RNN sequence length
        num_layers = 2 # number of layers in the RNN
        vocab_size = 65
        grad_clip = 5.
        if sample:
            print(">> sample mode:")
            batch_size = 1
            seq_length = 1
        # The core of the model consists of an LSTM cell that processes one char at a time and computes probabilities of the possible continuations of the char. 
        basic_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
        # model.cell.state_size is (128, 128)
        self.stacked_cell = tf.contrib.rnn.MultiRNNCell([basic_cell] * num_layers)

        self.input_data = tf.placeholder(tf.int32, [batch_size, seq_length], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, seq_length], name="targets")
        # Initial state of the LSTM memory.
        # The memory state of the network is initialized with a vector of zeros and gets updated after reading each char. 
        self.initial_state = stacked_cell.zero_state(batch_size, tf.float32) #why batch_size

        with tf.variable_scope('rnnlm_class1'):
            softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size]) #128x65
            softmax_b = tf.get_variable("softmax_b", [vocab_size]) # 1x65
            with tf.device("/cpu:0"):
                embedding = tf.get_variable("embedding", [vocab_size, rnn_size])  #65x128
                inputs = tf.split(tf.nn.embedding_lookup(embedding, self.input_data), seq_length, 1)
                inputs = [tf.squeeze(input_, [1]) for input_ in inputs]
                #inputs = tf.split(em, seq_length, 1)
                
                


        # The value of state is updated after processing each batch of chars.
        outputs, last_state = tf.contrib.legacy_seq2seq.rnn_decoder(inputs, self.initial_state, self.stacked_cell, loop_function=None, scope='rnnlm_class1')
        output = tf.reshape(tf.concat(outputs,1), [-1, rnn_size])
        self.logits = tf.matmul(output, softmax_w) + softmax_b
        self.probs = tf.nn.softmax(self.logits)
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([self.logits],
                [tf.reshape(self.targets, [-1])],
                [tf.ones([batch_size * seq_length])],
                vocab_size)
        self.cost = tf.reduce_sum(loss) / batch_size / seq_length
        self.final_state = last_state
        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),grad_clip)
        optimizer = tf.train.AdamOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
    
    
    def sample(self, sess, chars, vocab, num=200, prime='The ', sampling_type=1):
        state = sess.run(self.stacked_cell.zero_state(1, tf.float32))
        #print state
        for char in prime[:-1]:
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [state] = sess.run([self.final_state], feed)

        def weighted_pick(weights):
            t = np.cumsum(weights)
            s = np.sum(weights)
            return(int(np.searchsorted(t, np.random.rand(1)*s)))

        ret = prime
        char = prime[-1]
        for n in range(num):
            x = np.zeros((1, 1))
            x[0, 0] = vocab[char]
            feed = {self.input_data: x, self.initial_state:state}
            [probs, state] = sess.run([self.probs, self.final_state], feed)
            p = probs[0]

            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if char == ' ':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)

            pred = chars[sample]
            ret += pred
            char = pred
        return ret

### Creating the LSTM object
Now we create a LSTM model:

In [31]:
with tf.variable_scope("rnn"):
    model = LSTMModel()

# Train usinng LSTMModel class
We can train our model through feeding batches:

In [32]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs): # num_epochs is 5 for test, but should be higher
        sess.run(tf.assign(model.lr, learning_rate * (decay_rate ** e)))
        data_loader.reset_batch_pointer()
        state = sess.run(model.initial_state) # (2x[60x128])
        for b in range(data_loader.num_batches): #for each batch
            start = time.time()
            x, y = data_loader.next_batch()
            feed = {model.input_data: x, model.targets: y, model.initial_state:state}
            train_loss, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed)
            end = time.time()
        print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}" \
                .format(e * data_loader.num_batches + b, num_epochs * data_loader.num_batches, e, train_loss, end - start))
        with tf.variable_scope("rnn", reuse=True):
            sample_model = LSTMModel(sample=True)
            print sample_model.sample(sess, data_loader.chars , data_loader.vocab, num=50, prime='The ', sampling_type=1)
            print '----------------------------------'

ValueError: Could not flatten dictionary: key Tensor("MultiRNNCellZeroState/BasicRNNCellZeroState/zeros:0", shape=(60, 128), dtype=float32) is not unique.

## Want to learn more?

Running deep learning programs usually needs a high performance platform. PowerAI speeds up deep learning and AI. Built on IBM's Power Systems, PowerAI is a scalable software platform that accelerates deep learning and AI with blazing performance for individual users or enterprises. The PowerAI platform supports popular machine learning libraries and dependencies including Tensorflow, Caffe, Torch, and Theano. You can download a [free version of PowerAI](https://cocl.us/ML0120EN_PAI).

Also, you can use Data Science Experience to run these notebooks faster with bigger datasets. Data Science Experience is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, DSX enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of DSX users today with a free account at [Data Science Experience](https://cocl.us/ML0120EN_DSX)This is the end of this lesson. Hopefully, now you have a deeper and intuitive understanding regarding the LSTM model. Thank you for reading this notebook, and good luck on your studies.

### Thanks for completing this lesson!
Created by: <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>
This code is based on this [blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), and the code is an step-by-step implimentation of the [character-level implimentation](https://github.com/crazydonkey200/tensorflow-char-rnn).